In [3]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf 
import cv2

def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)


solve_cudnn_error()

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format


for i in range(1,20):
    filter_num = i*4
    ## Build the Convolution autoencoders model

    x = Conv2D(filter_num, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(filter_num, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    #x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    #encoded = MaxPooling2D((2, 2), padding='same')(x)


    x = Conv2D(filter_num, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filter_num, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    #x = Conv2D(16, (3, 3), activation='relu')(x)
    #x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)




    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    from keras.datasets import mnist
    import numpy as np

    (x_train, _), (x_test, _) = mnist.load_data()

    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

    from keras.callbacks import TensorBoard

    autoencoder.fit(x_train, x_train,
                                        epochs=20,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(x_test, x_test),
                                        callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])



print(autoencoder.summary())

'''
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(10)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
'''

#decoded_images = autoencoder.predict(x_test)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 17s 280us/step - loss: 0.1772 - val_loss: 0.1254
Epoch 2/20
60000/60000 [==============================] - 18s 292us/step - loss: 0.1154 - val_loss: 0.1069
Epoch 3/20
60000/60000 [==============================] - 22s 373us/step - loss: 0.1067 - val_loss: 0.1027
Epoch 4/20
59776/60000 [============================>.] - ETA: 0s - loss: 0.1013

KeyboardInterrupt: 